In [1]:
import pandas as pd
df=pd.read_csv('mon_data.csv')
df.dropna(subset=['text_image'],inplace=True)

In [2]:
df=df.iloc[:,[-1,3]]
df

,text_image,type
0,FACTURE\n\nLOGO\n\nJoanna Binet\n48 Coubertin\...,0
1,Joanna Binet\n48 Coubertin\n31400 Paris\n\nFAC...,0
2,FACTURE\n\nMon entreprise : Nom de la société\...,0
3,Joanna Binet\n48 Coubertin\n31400 Paris\n\nFAC...,0
4,Payer en ligne >\nFACTURE No\n\nSFIDELI\n\nF/0...,0
...,...,...
1603,Minois Retail Merchants Association\n\n36 Sout...,16
1604,FEMALE\n\n3\n2 :\n1\n\nMALE\n\n3-\n2\n\n3\nS\n...,20
1605,(B&W) PROTECTED BY MINNESOTA TOBACCO LITIGATIO...,14
1606,FF\n\nPrincipal Investigator/Program Director ...,11


In [3]:
import re
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
stop_words.update(["." , ",",':'])
stop_words=list(stop_words)
stop_words.extend(stopwords.words('french'))
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(str(w).lower().strip())
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!]+", " ", w)
    w = re.sub(r'\b\w{0,2}\b', '', w)

    # remove stopword
    mots = word_tokenize(w.strip())
    mots = [mot for mot in mots if mot not in stop_words]
    return ' '.join(mots).strip()



In [4]:
df.text_image = df.text_image.apply(lambda x :preprocess_sentence(x))


In [5]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     vector_size=200,
                     alpha=0.03, 
                     negative=10,
                     seed=1)


In [6]:
from nltk.corpus import stopwords
from nltk.tokenize.regexp import  RegexpTokenizer
stop_words = set(stopwords.words('french'))
stop_words.update(["." , ",",':'])
tokenizer=RegexpTokenizer(("[a-zA-Zé]{3,}"))
def stop_words_filtering(liste):
    liste_new=[]
    for i in liste:
        if i not in stop_words:
            liste_new.append(i)
        else:
            continue 
    return liste_new
sentences=[]
for txt in df.text_image:
    mots=stop_words_filtering(tokenizer.tokenize(str(txt).lower()))
    sentences.append(mots)

In [7]:
w2v_model.build_vocab(sentences, progress_per=100000)
words = list(w2v_model.wv.index_to_key)
print('Vocabulary size: %d' % len(words))
print(words)

Vocabulary size: 1106
['date', 'total', 'tobacco', 'new', 'research', 'net', 'source', 'nom', 'montant', 'non', 'university', 'salaire', 'cotisations', 'base', 'one', 'may', 'smoking', 'taux', 'valeur', 'cigarette', 'revenu', 'sociale', 'year', 'code', 'paie', 'name', 'health', 'facture', 'www', 'securite', 'cigarettes', 'heures', 'number', 'bulletin', 'csg', 'employeur', 'service', 'impot', 'paris', 'york', 'information', 'smoke', 'state', 'brut', 'company', 'payer', 'carte', 'imposable', 'use', 'american', 'page', 'per', 'study', 'rue', 'two', 'inc', 'time', 'program', 'brand', 'industrydocuments', 'https', 'chomage', 'adresse', 'paiement', 'also', 'tva', 'report', 'complementaire', 'data', 'contributions', 'years', 'national', 'part', 'assurance', 'medical', 'medicine', 'product', 'public', 'sante', 'would', 'francaise', 'telephone', 'retraite', 'general', 'smokers', 'department', 'please', 'high', 'amount', 'cancer', 'subject', 'form', 'office', 'description', 'address', 'client', 

In [8]:
v=[]
df_n=pd.DataFrame(columns=['text'])

In [9]:

for e,j in enumerate(df.text_image):
    mots=stop_words_filtering(tokenizer.tokenize(str(j).lower()))
    for i in mots:
        if i not in words:
             
            continue  
       
        else:
            v.append(i)
    df_n.loc[e]=' '.join(v)
            
    v=[]
    
            
#mot_vu=w2v_model.wv['facture']


In [10]:
df_n.text[3]
df.text_image.shape


(1566,)

In [11]:
import numpy as np
total_donnes=[]
text_total=np.zeros([200])
for txt in df_n.text:
    for mots in txt.split():
        text_total=text_total+w2v_model.wv[mots]
    total_donnes.append(text_total/len(txt.split()))
    text_total=np.zeros([200])
    
        

C:\Users\sadoun\AppData\Local\Temp\ipykernel_11980\1465323158.py:7: RuntimeWarning: invalid value encountered in true_divide
  total_donnes.append(text_total/len(txt.split()))


In [12]:
total_donnes=pd.DataFrame(np.array(total_donnes))
total_donnes['type']=df.type

In [13]:
total_donnes.shape
total_donnes.dropna(inplace=True)
total_donnes.head()

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,type
0,-0.001757,0.000317,-0.000249,0.000227,-0.000235,0.000390,-0.000087,0.001327,-0.000363,0.000073,...,-0.000311,0.000572,-0.000306,0.001215,-0.000892,0.000856,-0.000861,-0.000631,-0.000091,0.0
1,-0.001896,0.000027,-0.000242,0.000276,-0.000323,0.000511,-0.000262,0.001489,-0.000371,0.000160,...,-0.000183,0.000429,-0.000602,0.001549,-0.000563,0.000728,-0.000682,-0.000806,-0.000144,0.0
2,0.000157,0.001080,-0.000697,-0.000058,-0.000645,-0.000046,0.001049,0.001415,0.000278,0.000421,...,-0.000540,-0.000268,-0.001783,-0.000384,0.000113,-0.000458,-0.000062,0.000935,-0.000764,0.0
3,-0.001896,0.000027,-0.000242,0.000276,-0.000323,0.000511,-0.000262,0.001489,-0.000371,0.000160,...,-0.000183,0.000429,-0.000602,0.001549,-0.000563,0.000728,-0.000682,-0.000806,-0.000144,0.0
4,-0.000119,0.000430,-0.000700,0.000816,0.000739,0.001116,0.000147,0.001740,-0.000074,-0.000277,...,-0.000208,-0.000709,0.000095,0.000993,-0.000952,0.000984,-0.001158,0.000040,-0.000628,0.0


In [14]:
data=total_donnes.drop('type',axis=1)

target=total_donnes.type

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test=train_test_split(data,target,test_size=0.2,random_state=1234)

In [68]:
"""from sklearn import svm,model_selection,preprocessing
scaler=preprocessing.StandardScaler().fit(X_train)
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)"""

In [16]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D, Dropout,Activation
model=Sequential()
model.add(Dense(units = 256,input_shape=(200,)))
model.add((Activation('relu')))
model.add(Dropout(0.3))
model.add(Dense(units = 23, activation = "softmax"))
model.summary()
#clf=linear_model.LogisticRegression(C=)
#clf.fit(X_train,y_train)
#ac=AdaBoostClassifier(base_estimator=dtc,n_estimators=400)
#ac.fit(X_train, y_train)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               51456     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 23)                5911      
                                                                 
Total params: 57,367
Trainable params: 57,367
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(loss="sparse_categorical_crossentropy",optimizer = "adam", metrics = ["accuracy"])
model.fit(X_train,y_train,epochs = 50, batch_size = 32,validation_data = [X_test, y_test])

Epoch 1/50
33/33 [==============================] - 1s 11ms/step - loss: 3.0823 - accuracy: 0.2179 - val_loss: 2.9927 - val_accuracy: 0.2364
Epoch 2/50
33/33 [==============================] - 0s 3ms/step - loss: 2.8823 - accuracy: 0.2267 - val_loss: 2.7546 - val_accuracy: 0.2364
Epoch 3/50
33/33 [==============================] - 0s 3ms/step - loss: 2.6967 - accuracy: 0.2267 - val_loss: 2.6586 - val_accuracy: 0.2364
Epoch 4/50
33/33 [==============================] - 0s 3ms/step - loss: 2.6559 - accuracy: 0.2267 - val_loss: 2.6605 - val_accuracy: 0.2364
Epoch 5/50
33/33 [==============================] - 0s 3ms/step - loss: 2.6549 - accuracy: 0.2267 - val_loss: 2.6592 - val_accuracy: 0.2364
Epoch 6/50
33/33 [==============================] - 0s 3ms/step - loss: 2.6530 - accuracy: 0.2267 - val_loss: 2.6586 - val_accuracy: 0.2364
Epoch 7/50
33/33 [==============================] - 0s 3ms/step - loss: 2.6474 - accuracy: 0.2267 - val_loss: 2.6552 - val_accuracy: 0.2364
Epoch 8/50
33/33 [=

In [18]:
model.evaluate(X_test, y_test)

9/9 [==============================] - 0s 2ms/step - loss: 2.1970 - accuracy: 0.4922


[2.1969828605651855, 0.4922480583190918]